In [20]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import random 
from collections import defaultdict
import random
import scipy.io



In [25]:
#Setting a random seed
random.seed(1234)

#creating a dictionary based on the outputs.
def readoriginaldata():  
    data = np.loadtxt('./example_graphs/Douban/uir.index')
    return data    

c = readoriginaldata()


In [29]:
print c[1]


#Training Test Split
def train_test_split1(c, split):
    cTrain, cTest = train_test_split(c, test_size = split, random_state = 1)
    cTrain[:,1] = cTrain[:,1] + max(c[:,0]) 
    tempmax = max(c[:,0]) 
    np.savetxt('Ratings_Douban.Edgelist', cTrain, fmt='%6d')
    return cTrain, cTest, tempmax

cTrain, cTest, tempmax = train_test_split1(c, 0.1)



[ 0.  1.  4.]


In [38]:
#Embeddings generated by Deepwalk
def readEmbedData():  
    dataEmbed = pd.read_csv('./example_graphs/Douban/Ratings_Douban.embeddings')
    return dataEmbed

dataEmbed = readEmbedData()

#Creating a new Embedding matrix
def createDict(dataEmbed):
    dataEmbed = np.array(dataEmbed)
    dEmbed = [map(float, dataEmbed[i][0].split(' ')) for i in range(np.shape(dataEmbed)[0])]
    #print np.max(dEmbed)
    dEmbedDict = dict()
    for j in range(np.shape(dEmbed)[0]):
        if dEmbed[j][0] in dEmbedDict:
            dEmbedDict[dEmbed[j][0]].append(np.array(dEmbed[j][1:]))
        else:
            dEmbedDict[dEmbed[j][0]] = np.array(dEmbed[j][1:])          
    return dEmbedDict

vec = createDict(dataEmbed)

vecUsersMean = np.mean(dict((k, v) for k, v in vec.items() if k <= tempmax).values(),axis = 0)
vecMoviesMean = np.mean(dict((k, v) for k, v in vec.items() if k > tempmax).values(),axis = 0)
#print vecUsersMean, vecMoviesMean
#print np.inner(vecUsersMean, vecMoviesMean)



In [40]:
print vec[1]

[ 0.061441 -0.097592 -0.083157  0.080839  0.048947  0.222724 -0.173463
 -0.184141 -0.041812  0.013794 -0.077734  0.261048  0.130347 -0.003952
  0.459806  0.127724  0.041668  0.111716  0.252328  0.162133  0.026756
 -0.088197 -0.11856  -0.047901 -0.117794  0.116331  0.158517 -0.18027
  0.070207 -0.133399  0.081389 -0.012521 -0.055196  0.243452  0.110926
 -0.054581 -0.012021 -0.226002  0.208568 -0.101151  0.10229   0.138373
 -0.052834 -0.372677 -0.088053 -0.212646  0.053779  0.354005 -0.13642
 -0.418358 -0.184144 -0.064818  0.165919  0.184805 -0.026017 -0.028511
  0.047138  0.063213  0.213554  0.17831   0.211771  0.268317  0.179692
 -0.167824]


In [67]:
def evalRecoSys(vec, n, cTest, tempmax, vecMoviesMean, vecUsersMean):
    cTestMod = np.zeros((np.shape(cTest)[0], np.shape(cTest)[1]+1))
    cTestMod[:,:-1] = cTest
    for i in range(np.shape(cTest)[0]):
        userId = cTest[i][0]
        movieId = cTest[i][1] + tempmax
        if userId in vec:
            if movieId in vec:
                cTestMod[i][3] = np.inner(vec[userId], vec[movieId])*1.0#/(np.linalg.norm(vec[userId])*np.linalg.norm(vec[movieId])) 
            else:
                cTestMod[i][3] = np.inner(vec[userId], vecMoviesMean)*1.0#/(np.linalg.norm(vec[userId])*np.linalg.norm(vecMoviesMean))
        else:
            if movieId in vec:
                cTestMod[i][3] = np.inner(vecUsersMean, vec[movieId])*1.0#/(np.linalg.norm(vecUsersMean)*np.linalg.norm(vec[movieId]))
            else:    
                cTestMod[i][3] = np.inner(vecUsersMean, vecMoviesMean)*1.0#/(np.linalg.norm(vecUsersMean)*np.linalg.norm(vecMoviesMean))
    return cTestMod #count*1.0/np.shape(cTest)[0]

cTestMod = evalRecoSys(vec,1,cTest,tempmax,vecMoviesMean, vecUsersMean)

def precisionTopn(cTestMod,n):
    dictTopn = defaultdict(list)
    for i in range(np.shape(cTestMod)[0]):
        if cTestMod[i][0] not in dictTopn.keys():
            dictTopn[cTestMod[i][0]] = []
            dictTopn[cTestMod[i][0]].append(list(cTestMod[i][1:]))
        else:
            dictTopn[cTestMod[i][0]].append(list(cTestMod[i][1:]))
    correct = 0.0
    incorrect = 0.0
    count = 0.0
    print np.shape(dictTopn.keys())
    for j in dictTopn.keys():        
        if np.shape(dictTopn[j])[0]>=n:
            count += 1
            arr = np.array(dictTopn[j][:])
            correct += np.sum(arr[arr[:, 2].argsort()][-n:,1] >= arr[arr[:, 1].argsort()][-n][1])
            incorrect += np.sum(arr[arr[:, 2].argsort()][-n:,1] < arr[arr[:, 1].argsort()][-n][1])         
    print count, correct/(correct+incorrect)    
    return dictTopn,correct, incorrect, count
                

def recallTopn(cTestMod, n):
    dictRTopn = defaultdict(list)
    for i in range(np.shape(cTestMod)[0]):
        if cTestMod[i][0] not in dictRTopn.keys():
            dictRTopn[cTestMod[i][0]] = []
            dictRTopn[cTestMod[i][0]].append(list(cTestMod[i][1:]))
        else:
            dictRTopn[cTestMod[i][0]].append(list(cTestMod[i][1:]))
    correct = 0.0
    incorrect = 0.0
    count = 0.0
    print np.shape(dictRTopn.keys())
    for j in dictRTopn.keys():        
        if np.shape(dictRTopn[j])[0]>n:
            count += 1
            arr =np.array(dictRTopn[j][:])
            randItem = arr[arr[:, 1].argsort()][-random.randint(1,n)]
            if randItem[0] in arr[arr[:, 2].argsort()][-n:,0]:
                correct += 1
            else:
                incorrect += 1
    print count, correct/(correct+incorrect)
    return dictRTopn, correct, incorrect, count
#cTrainT = cTrain[:,np.argsort([1,0,2])]

#cTrainComb = np.vstack((cTrain, cTrainT))
#print cTrainT, np.shape(cTrain), np.shape(cTrainT), np.shape(cTrainComb)





In [74]:
import time
A = time.time()
print np.shape(cTestMod)
A1, B1, C1, D1 = precisionTopn(cTestMod, 10)
print time.time()-A


(3366168, 4)


KeyboardInterrupt: 

In [15]:
import numpy as np
import scipy.sparse as sps


graph = sps.csr_matrix((cTrain[:,2], (cTrain[:,0], cTrain[:,1])))
U,S,V = sps.linalg.svds(graph, k=64, ncv=None, tol=0, which='LM', v0=None, maxiter=None, return_singular_vectors=True)
print np.shape(U),S,np.shape(V)



(49290, 64) [  86.41751891   86.44999022   86.66401734   87.16972521   87.27506927
   87.70086099   88.68576352   88.78484946   89.32399124   89.70169606
   89.74448259   90.14149118   90.51375379   90.90214914   91.64072228
   92.17345468   92.36502059   93.00035841   93.44307614   93.94983177
   94.46264589   94.66194022   95.6131031    95.97049342   96.31529322
   97.12219602   97.22916899   97.97854724   98.45640513   99.38784468
  100.03791472  101.05766151  101.75067026  102.04597065  103.45954
  104.4934357   104.90958051  105.56121786  106.99121026  107.32720614
  107.97478462  108.35900228  108.96699096  110.03830179  110.56699983
  112.1768973   113.01154894  113.16180654  113.77694309  114.26748824
  115.49381208  116.64140562  117.42250821  118.34792205  121.5048287
  125.00086013  127.69910021  129.25510137  131.47204561  135.85640421
  138.87413838  156.51590803  189.32208915  260.76497968] (64, 189027)


In [50]:
def evalRecoSysSVD(U, S, V, cTest, tempmax):
    cTestMod = np.zeros((np.shape(cTest)[0], np.shape(cTest)[1]+1))
    cTestMod[:,:-1] = cTest
    for i in range(np.shape(cTest)[0]):
        userId = cTest[i][0]
        movieId = cTest[i][1]
        cTestMod[i][3] = np.inner(U[userId,:], V[:,movieId])*1.0 #/(np.linalg.norm(U[userId,:])*np.linalg.norm(V[:,movieId])) 
        #if userId in vec:
        #    if movieId in vec:
        #        cTestMod[i][3] = np.inner(U[userId,:], V[:,movieId])*1.0/(np.linalg.norm(U[userId,:])*np.linalg.norm(V[:,movieId])) 
        #    else:
        #        cTestMod[i][3] = np.inner(vec[userId], vecMoviesMean)*1.0/(np.linalg.norm(vec[userId])*np.linalg.norm(vecMoviesMean))
        #else:
        #    if movieId in vec:
        #        cTestMod[i][3] = np.inner(vecUsersMean, vec[movieId])*1.0/(np.linalg.norm(vecUsersMean)*np.linalg.norm(vec[movieId]))
        #    else:    
        #        cTestMod[i][3] = np.inner(vecUsersMean, vecMoviesMean)*1.0/(np.linalg.norm(vecUsersMean)*np.linalg.norm(vecMoviesMean))
    return cTestMod #count*1.0/np.shape(cTest)[0]

cTestMod = evalRecoSysSVD(U, S, V, cTest, tempmax)

/Users/shahv1/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [51]:
AA1, BB1, CC1, DD1 = precisionTopn(cTestMod, 1)
AA3, BB3, CC3, DD3 = precisionTopn(cTestMod, 3)
AA5, BB5, CC5, DD5 = precisionTopn(cTestMod, 5)
AA10, BB10, CC10, DD10 = precisionTopn(cTestMod, 10)

RR1, SS1, TT1, UU1 = recallTopn(cTestMod, 1)
RR3, SS3, TT3, UU3 = recallTopn(cTestMod, 3)
RR5, SS5, TT5, UU5 = recallTopn(cTestMod, 5)
RR10, SS10, TT10, UU10 = recallTopn(cTestMod, 10)

17948.0 0.561845330956
10081.0 0.645537810402
6299.0 0.680139704715
2664.0 0.728378378378
17948.0 0.381100958324
10081.0 0.520682471977
6299.0 0.567074138752
2664.0 0.614489489489


In [ ]:
def matrixfactorization(vec, tempmax, vecMoviesMean, vecUsersMean, iter_n, cTrain):
    while i < range(10*np.shape(cTrain)[0]):
        randomId = random.randint(1,np.shape(cTrain)[0])
        userId = cTrain[randomId][0]
        movieId = cTrain[randomId][1] + tempmax
        vec
        
        
        
        
        

In [ ]:
#Extra functions   
#Finding graph statistics
def degreeStats(cTrain):
    size = np.shape(cTrain)[0]
    cTrain.astype(int)
    nodes = np.zeros((size,1))
    for i in range(size):
        nodes[cTrain[i][0]] += cTrain[i][2]
        nodes[cTrain[i][1]] += cTrain[i][2]
    nodes = nodes/2.0
    nodesMean = np.mean(nodes)
    nodesVar = np.var(nodes)
    return  nodesMean, nodesVar

avgDegree, varDegree = degreeStats(cTrain)
        
    
def countTestSet(cTestMod, tempmax):
    countUser = np.zeros(tempmax+1)
    for i in range(np.shape(cTestMod)[0]):
        countUser[cTestMod[i][0]] += 1
    return countUser

Out = countTestSet(cTestMod, tempmax)    
print np.sum(Out>5)
print cTestMod

In [47]:
randomId = random.randint(1,np.shape(cTrain)[0])
userId = cTrain[randomId][0]
print userId

37799.0
